In [310]:
import urllib.request as urllib
import json
import csv
htmltext = urllib.urlopen("https://fantasy.premierleague.com/drf/element-summary/1")
data = json.load(htmltext)
scoredata = data['history']
keys = scoredata[0].keys()
output_file=open('players.csv', 'w')
dict_writer = csv.DictWriter(output_file, keys)
dict_writer.writeheader()
for i in range(1,567):
    htmltext = urllib.urlopen("https://fantasy.premierleague.com/drf/element-summary/"+str(i))
    data = json.load(htmltext)
    scoredata = data['history']
    keys = scoredata[0].keys()
    dict_writer.writerows(scoredata)
output_file.close()    

In [411]:
import pandas as pd
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import train_test_split
df = pd.read_csv('player.csv')
X = df.loc[((df['round']>0)&(df['round']<11))]
y = df.loc[((df['round']>1)&(df['round']<12))]['total_points']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)


In [434]:
from sklearn.ensemble import GradientBoostingClassifier

model2 = GradientBoostingClassifier()
model2.fit(X_train, y_train)
print(model2.score(X_test, y_test))
Bag = BaggingClassifier(base_estimator = model2, max_features = 0.7, max_samples = 0.7)
Bag.fit(X_train, y_train)
print(Bag.score(X_train, y_train))
print(Bag.score(X_test, y_test))



0.544600938967
0.833768267223
0.546165884194


In [440]:
from sklearn import metrics
import pickle
y_pred=Bag.predict(X_train)
print(np.sqrt(metrics.mean_squared_error(y_train, y_pred)))
y_pred=Bag.predict(X_test)
print(np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
pickle.dump(Bag, open('model', 'wb'))

1.26777549167
2.54889586087


In [451]:
df = pd.read_csv('player2.csv')
x_valid= df.loc[(df['round'] == 11)]
model = pickle.load(open('model', 'rb'))


In [481]:
#get user input after conversion to id
X_valid=x_valid.loc[(x_valid['element'] == 234)]
res=model.predict(x_valid)
res

array([ 0.,  1.,  0.,  6.,  0.,  0.,  1.,  3.,  0.,  6.,  0.,  0.,  1.,
        2.,  0.,  2.,  0.,  3.,  2.,  1.,  3.,  0.,  1.,  1.,  1.,  0.,
        0.,  2.,  2.,  0.,  0.,  2.,  2.,  2.,  1.,  0.,  0.,  2.,  3.,
        3.,  2.,  0.,  0.,  0.,  2.,  0.,  0.,  1.,  0.,  0.,  2.,  0.,
        0.,  1.,  2.,  1.,  0.,  2.,  1.,  0.,  0.,  2.,  2.,  0.,  0.,
        1.,  0.,  0.,  0.,  2.,  2.,  1.,  0.,  0.,  0.,  2.,  2.,  2.,
        2.,  0.,  0.,  0.,  0.,  2.,  1.,  2.,  2.,  0.,  0.,  2.,  1.,
        2.,  1.,  1.,  0.,  0.,  1.,  2.,  2.,  1.,  0.,  0.,  1.,  2.,
        5.,  1.,  1.,  1.,  0.,  0.,  2.,  0.,  0.,  0.,  0.,  2.,  0.,
        1.,  1.,  0.,  0.,  0.,  1.,  0.,  0.,  2.,  0.,  0.,  1.,  0.,
        2.,  2.,  0.,  0.,  1.,  0.,  0.,  2.,  0.,  2.,  0.,  0.,  0.,
        0.,  0.,  1.,  0.,  0.,  0.,  0.,  2.,  0.,  1.,  0.,  2.,  2.,
        1.,  0.,  2.,  0.,  2.,  0.,  2.,  0.,  0.,  0.,  0.,  1.,  0.,
        0.,  1.,  2.,  2.,  2.,  0.,  0.,  0.,  0.,  0.,  0.,  1

In [505]:


firstName = "Mohamed"
lastName = "Salah"
i=234

htmltext = urllib.urlopen("https://fantasy.premierleague.com/drf/bootstrap-static")
input_json = json.load(htmltext)
elements = input_json["elements"]
out_dir={"element":[]}
for i in range(1,560):
    playerDict = [x for x in elements if (x["id"] == i)]
    name = playerDict[0]["first_name"]
    secondName=playerDict[0]["second_name"]
    X_valid=x_valid.loc[(x_valid['element'] == i)]
    res=model.predict(X_valid)[0]
    gs=X_valid['goals_scored'].values[0]
    assists=X_valid['assists'].values[0]
    cs=X_valid['clean_sheets'].values[0]
    minutes=X_valid['minutes'].values[0]
    PrevPoints=X_valid['total_points'].values[0]
    out_dir["element"].append({"full_name":name+" "+secondName,"previous_points":PrevPoints,"predicted_points":res,"minutes_played":minutes,"goals_scored":gs,"assists":assists,"clean_sheets":cs})
with open('result.json', 'w') as fp:
    json.dump(out_dir, fp)